In [1]:
import panel as pn
pn.extension() # loading panel's extension for jupyter compatibility 
text_input = pn.widgets.TextInput()

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from random import choice

In [3]:
tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [4]:
def get_pred(text, model, tok, p=0.7):
    input_ids = torch.tensor(tok.encode(text)).unsqueeze(0)
    logits = model(input_ids)[0][:, -1]
    probs = F.softmax(logits, dim=-1).squeeze()
    idxs = torch.argsort(probs, descending=True)
    res, cumsum = [], 0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum > p:
            pred_idx = idxs.new_tensor([choice(res)])
            break
    pred = tok.convert_ids_to_tokens(int(pred_idx))
    return tok.convert_tokens_to_string(pred)

In [5]:
bmp=pn.panel('https://authorsunite.com/wp-content/uploads/2020/05/ghostwriting.png',width=100)

In [6]:
generated_text = pn.pane.Markdown(object=text_input.value)


In [7]:
text_input.link(generated_text, value='object')


Watcher(inst=TextInput(), cls=<class 'panel.widgets.input.TextInput'>, fn=<function Reactive.link.<locals>.link at 0x00000200C6338168>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

In [8]:
button = pn.widgets.Button(name="Generate",button_type="primary")


In [9]:
def click_cb(event):
    for i in range(10):
        pred =get_pred(generated_text.object, model, tok)
        generated_text.object += pred

In [10]:
button.on_click(click_cb)


In [11]:
app = pn.Column(bmp,text_input, button, generated_text); 


In [12]:
title = pn.pane.Markdown("# **Ghost Writer**")
desc = pn.pane.HTML("<marquee scrollamount='10'><b>Welcome to Ghost writer! In order to get started, simply enter some starting input text below, click generate a few times and watch it go!</b></marquee>")
final_app = pn.Column(title, desc ,app)


In [13]:
final_app.show()

Launching server at http://localhost:60418


In [16]:
final_app.servable()



Column
    [0] Markdown(str)
    [1] HTML(str)
    [2] Column
        [0] TextInput()
        [1] Button(button_type='primary', name='Generate')
        [2] Markdown(str)